<!-- <img src="media/titlepage.png" width="100%"> -->


# Tutorial on Julia and Trixi.jl

## Brief introduction to Julia

### Hendrik Ranocha, Michael Schlottke-Lakemper

Follow along at https://github.com/trixi-framework/tutorial-2023-kassel

# Brief introduction to Julia

[Julia](https://julialang.org) is a modern high-level programming language developed specifically with scientific computing in mind. We will briefly introduce Julia and demonstrate some of its design principles to help you getting started with the tutorial on [Trixi.jl](https://github.com/trixi-framework/Trixi.jl), our framework of high-order methods for hyperbolic PDEs written in Julia. This introduction is aimed at researchers in numerical analysis with previous programming experience.


## Further information on running this notebook

This introduction is available as a Jupyter notebook at https://github.com/trixi-framework/tutorial-2023-kassel, including information how to set up everything. For more information about Trixi and how to use it, please visit [Trixi on GitHub](https://github.com/trixi-framework/Trixi.jl) or refer to the [official documentation](https://trixi-framework.github.io/Trixi.jl/stable/).

This notebook was set up and tested with Julia v1.9.0 but may also work with other versions.

*Note:* If you change a variable in a later cell and then re-execute an earlier cell, the results might change unexpectedly. Thus if in doubt, re-run the entire notebook *in order*. The reason is that all cells in a Jupyter notebooks share a common variable space.

*Note:* This notebook is tested using Firefox. Most parts should also work for other browsers such as Chrome, but the videos used in the last demonstrations might not be displayed correctly.


## Authors and license

This material is distributed by Hendrik Ranocha and Michael Schlottke-Lakemper under the MIT license. It is inspired by and partially derived from the talks
- [Robin Deits (2020), Intro to Julia Programming Language with Detroit Tech Watch](https://www.youtube.com/watch?v=qLO-yaUkLKE)
- [Hendrik Ranocha (2021), Introduction to Julia and Trixi, a numerical simulation framework for hyperbolic PDEs](https://github.com/trixi-framework/talk-2021-Introduction_to_Julia_and_Trixi)

In [ ]:
# Install all dependencies used in this talk
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

In [ ]:
ENV["COLUMNS"] = 100 # display width

# Introduction to Julia

- [julialang.org](https://julialang.org)
- Julia is a high-level language like Python/Matlab with the performance of a fast language like C/C++/Fortran
- Julia is designed for scientific computing...
  - N-dimensional arrays
  - Reproducibility
- ...and valuable for general programming
  - Growing ecosystem of packages
  - Rich type system
- Encourages good software development practices

## Julia at a glance

- First public release in 2012, version 1.0 released in 2018
- Free and open source (Julia itself is MIT licensed)
- Built-in JIT compiler transforms Julia code to native assembly at run time
  - Uses LLVM under the hood
- Garbage collected
- Dynamically typed
- Organized via multiple dispatch

## A brief tour of Julia

### The basics

In [ ]:
# Arithmetic
1 + 2

In [ ]:
# Strings
println("Hello world")

In [ ]:
# Arrays
x = [1, 2, 3]
sum(x)

### Unicode and LaTeX

In [ ]:
# type `\beta` + TAB
β = π / 4
tan(β) ≈ sin(β) / cos(β)

In [ ]:
using LinearAlgebra, Plots, LaTeXStrings
n = 1_000
λ = eigvals(randn(n, n))
scatter(real(λ), imag(λ), aspect_ratio = :equal, legend = nothing,
        xguide = L"\operatorname{Re} \lambda", yguide = L"\operatorname{Im} \lambda")

In [ ]:
# to free some memory on mybinder.org
λ = nothing
GC.gc()

### Sparse linear algebra

In [ ]:
using SparseArrays
n = 30
A = spdiagm(0 => 2 * ones(n), 1 => -ones(n-1), -1 => -ones(n-1))

In [ ]:
f = rand(n)
u = A \ f
norm(A * u - f)

### Functions

In [ ]:
function say_hello(to_whom)
    println("Hello ", to_whom)
end

In [ ]:
say_hello("world")

Functions are generic, so you can pass everything that works (duck typing)

In [ ]:
say_hello([1, 2, 3])

### Types

Everything in Julia has a type

In [ ]:
typeof(1)

In [ ]:
typeof(1.0)

In [ ]:
typeof(π)

In [ ]:
typeof(typeof)

In [ ]:
typeof([1, 2, 3])

You can create your own types easily

In [ ]:
struct Person
    name::String
end

alice = Person("Alice")

User-defined types are as efficient as anything built-in:

In [ ]:
struct Point2D
    x::Float64
    y::Float64
end

p = Point2D(1.0, 2.0)

sizeof(p)

### Multiple dispatch (short version)

Julia implements the paradigm of "dynamic multiple dispatch" (or, abbreviated, just "multiple dispatch"). Multiple dispatch is similar to what is known from other programming languages as *function overloading*: You can have multiple functions with the same name, as long as they differ in their call signature, i.e., if they have different argument types and/or a different number of arguments. In contrast to compiled languages such as C/C++ or Fortran, this works not just with types known statically to the compiler, but also with data where the concrete type is only known at *runtime*.

For a more in-depth introduction to multiple dispatch, see [Stefan Karpinski's talk at JuliaCon (2019)](https://www.youtube.com/watch?v=kc9HwsxE1OY).

We start by creating a generic version of a function to calculate the volume integral in a numerical simulation, called `calc_volume_integral!`. The `!` is just a regular character here and is a best practice convention in Julia to denote a function that mutates at least one of its inputs. Since we do not care about computing anything here, we just have it print a short message:

In [ ]:
calc_volume_integral!(du, u, mesh, equations) = println("general version")

As a next step, we create one data type for the mesh and two data types for the `equations` argument, and create an instance for each:

In [ ]:
struct StructuredMesh end
struct CompressibleEulerEq end
struct NavierStokesEq end

mesh = StructuredMesh()
euler_eq = CompressibleEulerEq()
navier_stokes_eq = NavierStokesEq()

Now we can start dispatching! We create two additional **methods** for our function, one for each of our new equation types:

In [ ]:
calc_volume_integral!(du, u, mesh, equations::CompressibleEulerEq) = println("Compressible Euler version")
calc_volume_integral!(du, u, mesh, equations::NavierStokesEq) = println("Navier-Stokes version")

We can now have a look at what is happening when calling `calc_volume_integral!` with different arguments. For this, we create some dummy argumemnts for `du` and `u`, and then call `calc_volume_integral!` with the variable of type `CompressibleEulerEquations`:

In [ ]:
du = rand(5); u = rand(5)

calc_volume_integral!(du, u, mesh, euler_eq)

As expected, the compressible Euler specialization was used. Similarly, if we call the function with the Navier-Stokes variable, we get:

In [ ]:
calc_volume_integral!(du, u, mesh, navier_stokes_eq)

And finally, if we use an argument of a type for which no special function is available, we get a call to the original, generic version:

In [ ]:
struct AdvectionEquation end
advection_eq = AdvectionEquation()

calc_volume_integral!(du, u, mesh, advection_eq)

This was dispatch on a single input. We now create a new mesh type for adaptive meshes:

In [ ]:
struct AdaptiveMesh end
adaptive = AdaptiveMesh()

Then we implement a corresponding specialization that dispatches on *both* the mesh and the equations type:

In [ ]:
function calc_volume_integral!(du, u, mesh::AdaptiveMesh, equations::NavierStokesEq)
    println("Navier-Stokes for adpative meshes version")
end

As expected, we still get the general NSE version if we use our structured mesh variable:

In [ ]:
calc_volume_integral!(du, u, mesh, navier_stokes_eq)

However, when we use our new, adaptive mesh type as input, we get the specialized version:

In [ ]:
calc_volume_integral!(du, u, adaptive, navier_stokes_eq)

All of this could theoretically have been achieved using static multiple dispatch a.k.a. as function overloading. However, what does it mean that Julia supports dynamic multiple dispatch? In essence, it means that this concept of using the most "fitting" method will still work, even if the compiler is **fundamentally unable** to determine the types of the input arguments ahead of time.

To illustrate this, we will have a look at the following function. It sets up a dummy numerical simulation and then decides *randomly* whether to create a structured mesh or an adaptive mesh (whether this is good coding practice, we will leave up to the audience to decide...):

In [ ]:
function run_simulation()
    du = rand(5)
    du = rand(5)
    
    equations = NavierStokesEq()
    
    if rand(Bool)
        mesh = AdaptiveMesh()
    else
        mesh = StructuredMesh()
    end
    
    calc_volume_integral!(du, u, mesh, equations)
end

When we now run the simulation multiple times, we can see that depending on the *type* of the randomly generated mesh, the correct version of `calc_volume_integral!` is called, even though it is impossible for the compiler to know the types beforehand:

In [ ]:
for i in 1:10
    run_simulation()
end

Finally, what do you get out of this? First, it allows one to forget about the distinction between runtime types and compile-time types during code development: You can be sure, that Julia will always dispatch on the actual type and do The Right Thing™.

In addition, however, multiple dynamic dispatch is one important component of what enables the easy interoperability between packages. The Julia package ecosystem if built around types, and if you write your package code in a generic way, others can easily extend it for their own types.

For example, we can extend the method `Base.abs`, which returns the absolute value of its argument, to work with our newly created type `Point` from the previous section:

In [ ]:
# struct Point2D
#     x::Float64
#     y::Float64
# end
Base.abs(p::Point2D) = Point2D(abs(p.x), abs(p.y))

This implementation will do the (somewhat) sane thing here and call `abs` on each of the `Point2D`'s fields. We can now see this in action by creating a point with negative entries and then feeding it through `abs`:

In [ ]:
p = Point2D(-2.0, 5.0)

abs(p)

Note that this is not operator overloading, and that due to just-in-time compilation, the performance of this example code is just as good as for built-in Julia types.

### Multiple dispatch (extended version)

Julia does not use classes to organize nouns (types) and verbs (functions). Instead, multiple dispatch is a central design decision. Thus, the compiler chooses an appropriate method of a given function based on the types of all arguments (not their values!).

For more information, see [Stefan Karpinski's talk at JuliaCon (2019)](https://www.youtube.com/watch?v=kc9HwsxE1OY).

In [ ]:
greet(x, y) = println(x, " greets ", y)

In [ ]:
alice = Person("Alice")
bob = Person("Bob")

greet(alice, bob)

Currently there is only one greet() function, and it will work on `x` and `y` of any type:

In [ ]:
greet(π, "Kassel")

We can use abstract types to organize the behavior of related types:

In [ ]:
abstract type Animal end

struct Cat <: Animal
    name::String
end

We've already defined `greet(x, y)` for any `x` and `y`, but we can add another definition for a more specific set of input types.

We can be as specific or as general as we like with the argument types:

In [ ]:
greet(x::Person, y::Animal) = println(x, " pats ", y)

In [ ]:
greet(x::Cat, y) = println(x, " meows at ", y)

Julia will always pick the *most specific* method that matches the provided function arguments.

In [ ]:
fluffy = Cat("Fluffy")

greet(alice, fluffy)

In [ ]:
greet(fluffy, alice)

In [ ]:
struct Dog <: Animal
    name::String
end

greet(x::Dog, y) = println(x, " barks at ", y)

greet(x::Dog, y::Person) = println("$x licks $y's face")

greet(x::Dog, y::Dog) = println("$x sniffs $y's butt")

In [ ]:
fido = Dog("Fido")
rex = Dog("Rex")

greet(alice, fido)

In [ ]:
greet(fido, fluffy)

In [ ]:
greet(fido, bob)

In [ ]:
greet(fido, rex)

If you want to know which `greet` method will be called for a given set of arguments, you can use `@which` to check:

In [ ]:
@which greet(alice, fido)

You can list all of the methods of a given function with `methods`:

In [ ]:
methods(greet)

You can access docstrings using `?`.

In [ ]:
?methods

Tab completion works in Julia and can print possible signatures of functions.

In [ ]:
greet( # type TAB

## Modules

Modules in Julia are used to organize code into namespaces.

In [ ]:
module MyUsefulModule

export hello

hello()   = println("Hello world")
goodbye() = println("Goodbye world")

end

MyUsefulModule.hello()

The `using` command brings any `export`ed symbols from a module into the current namespace:

In [ ]:
using .MyUsefulModule
hello()

## Using packages

Julia has a built-in package manager called `Pkg`. It handles installing packages and managing all your package environments. 

A package *environment* represents a single set of installed packages. Let's activate the environment for this talk:

In [ ]:
using Pkg
Pkg.activate(".")
Pkg.instantiate()

(this is similar to `source venv/bin/activate` in a Python virtual environment)

We can install a package in our current environment. This will only affect that environment, so we can safely do this without breaking any other Julia projects we might be working on:

In [ ]:
Pkg.add("BenchmarkTools")

The `Project.toml` file gives a concise description of the packages we've added to this environment:

In [ ]:
run(`cat Project.toml`)

The package manager also generates a complete manifest of every package that is installed, including all the transitive dependencies and their versions. You can use this to reproduce a given package environment exactly:

In [ ]:
run(`head -n 20 Manifest.toml`)

# Julia is fast

* I claimed at the beginning of this talk that Julia has performance on par with C. Let's prove it!
* To show this, I'll implement the basic `sum` function in Julia, C, and Python so we can compare them:

Let's start with Julia:

In [ ]:
"""
    my_sum(x)

Naive implementation of sum. Works for any iterable `x` with any element type.
"""
function my_sum(x)
    result = zero(eltype(x))
    for element in x
        result += element
    end
    return result
end

And let's create some data to test with:

In [ ]:
data = randn(Float64, 10^7);

To measure the performance of `my_sum`, we'll use the [BenchmarkTools](https://github.com/JuliaCI/BenchmarkTools.jl) package. 

In [ ]:
using BenchmarkTools

In [ ]:
@benchmark my_sum($data)

In this case, we only care about the minimum time. The `@btime` macro is a shorthand to print just that minimum time:

In [ ]:
@btime my_sum($data)

Let's compare this with C. It's easy to call functions from C shared libraries in Julia:

In [ ]:
"""
Call the `strcmp` function from `libc.so.6`
"""
function c_compare(x::String, y::String)
    # We have to tell the compiler that this C function returns an `int` and 
    # expects two `char *` inputs. The `Cint` and `Cstring` types are convenient
    # shorthands for those:
    ccall((:strcmp, "libc.so.6"), Cint, (Cstring, Cstring), x, y)
end

In [ ]:
c_compare("hello", "hello")

Calling C functions has very little overhead:

In [ ]:
@btime c_compare($("hello"), $("hello"))

Let's create a C implementation of `my_sum`. We can do that without leaving Julia by piping some code directly to GCC:

In [ ]:
C_code = """
#include <stddef.h>  // For `size_t`

// Note: our Julia code works for any type, but the C implementation 
// is only for `double`.

double c_sum(size_t n, double* x) {
    double s = 0.0;
    size_t i;
    for (i = 0; i < n; ++i) {
        s += x[i];
    }
    return s;
}
""";

Now let's generate a name for our shared library:

In [ ]:
# dlext gives the correct file extension for a shared library on this platform
using Libdl: dlext
Clib = tempname() * "." * dlext

To send the code to GCC, we can use `open()` on a command to write directly to the `stdin` of that command as if it were any other file- or buffer-like object:

In [ ]:
open(`gcc -fPIC -O3 -march=native -mtune=native -xc -shared -o $Clib -`, "w") do cmd
    print(cmd, C_code) 
end

Now we can define a Julia function that calls the C function we just compiled:

In [ ]:
# The return type and argument types must match the signature we declared above:
# 
#   double c_sum(size_t n, double* x) 
# 
c_sum(x::Array{Float64}) = @ccall Clib.c_sum(length(x)::Csize_t, x::Ptr{Cdouble})::Cdouble

Now let's measure the performance of the pure C function:

In [ ]:
@btime c_sum($data)

Let's plot the result using the [Plots](https://github.com/JuliaPlots/Plots.jl) package.

In [ ]:
using Plots

In [ ]:
results = Dict(
    "my_sum (Julia)" => 10.3,
    "c_sum (C)" => 10.3
)

xlabel="function"; ylabel="time (ms, shorter is better)"; legend=nothing;
bar(results; xlabel, ylabel, legend)

Our naive Julia code is just as fast as our naive C code! 

Is that as fast as we can go? What about Julia's built-in `sum()` function:

In [ ]:
@btime sum($data)

In [ ]:
results["sum (Julia)"] = 5.2

bar(results; xlabel, ylabel, legend)

What's going on? Is the `sum()` function using some built-in behavior we don't have access to?

Nope - we can achieve that result easily with a few modifications:

In [ ]:
function my_fast_sum(x)
    result = zero(eltype(x))

    # `@simd` enables additional vector operations by indicating that it is OK to potentially
    # evaluate the loop out-of-order. 
    @simd for element in x
        result += element
    end
    result
end

In [ ]:
@btime my_fast_sum($data)

In [ ]:
results["my_fast_sum (Julia)"] = 5.3

bar(results; xlabel, ylabel, legend)

With some pretty simple changes, we were able to create a pure-Julia function which is twice as fast as our naive C function while still being clear and completely generic:

In [ ]:
my_fast_sum([1, 2.5, π])

Just for reference, let's compare with Python. It's easy to call Python code from Julia too - we just need the [PyCall](https://github.com/JuliaPy/PyCall.jl) package (or the newer alternative [PythonCall](https://github.com/cjdoris/PythonCall.jl)):

In [ ]:
using PyCall

In [ ]:
py_math = pyimport("math")
py_math.sin.([0, π/2, π])

Just as we did with C, we can quickly define a Python sum function without leaving Julia:

In [ ]:
# The PyCall package lets us define python functions directly from Julia:

py"""
def mysum(a):
    s = 0.0
    for x in a:
        s = s + x
    return s
"""

# mysum_py is a reference to the Python mysum function
py_sum = py"""mysum"""o

Let's make sure we're getting similar answers everywhere:

In [ ]:
py_sum(data) ≈ c_sum(data) ≈ sum(data) ≈ my_sum(data) ≈ my_fast_sum(data)

In [ ]:
@btime py_sum($data)

In [ ]:
results["py_sum (Python)"] = 1.4e3

bar(results; xlabel, ylabel, legend)

### What about Numpy or Cython?

* Of course, there are faster ways to sum a vector of `double`s in Python than a `for` loop. 
* `numpy.sum()` is just as fast as Julia's `sum()` for large vectors...
* ...but there are some caveats:
  * NumPy is only efficient for a pre-determined set of numeric types. 
  * NumPy cannot be extended without switching into an entirely different programming language, build system, and code environment. 
  * So, if `numpy.sum()` happens to cover the cases you actually need, then go for it!
  * But if you want to be able to write efficient code that does not happen to cover the specific set of functions and types in NumPy, then you need something else. 

In [ ]:
struct Point{T}
    x::T
    y::T
end

function Base.zero(::Type{Point{T}}) where {T} 
    Point{T}(zero(T), zero(T))
end
    
Base.:+(p1::Point, p2::Point) = Point(p1.x + p2.x, p1.y + p2.y)

n = 10^7; points = Point.(randn(n), randn(n));

In [ ]:
@btime my_fast_sum($points)

In [ ]:
@code_native debuginfo=:none my_fast_sum(points)

# Bonus features of Julia

## Anything can be a value

Julia has no special rules about what can or cannot be assigned to a variable or passed to a function. 

### Functions are values

A Julia function is a value like any other, so passing functions around and implementing higher-order functions is trivial. This approach is used in several higher order functions of Julia such as `mapreduce`.

In [ ]:
data = randn(10^4)
sum(data) ≈ mapreduce(identity, +, data)

Functions can be inlined, even into standard library code. The compiler heuristics are often good and you can nudge it using `@inline` if necessary. Let's measure the performance of `mapreduce` with out own functions using `@benchmark` from BenchmarkTools:

In [ ]:
using BenchmarkTools

my_identity(x) = x
my_plus(x, y) = x + y
@benchmark mapreduce($my_identity, $my_plus, $data)

This is similar to using the built-in function `sum`

In [ ]:
@benchmark sum($data)

If we tell the compiler not to inline our functions, the performance will be reduced significantly, of course.

In [ ]:
@noinline my_identity_not_inlined(x) = x
@noinline my_plus_not_inlined(x, y) = x + y
@benchmark mapreduce($my_identity_not_inlined, $my_plus_not_inlined, $data)

In [ ]:
# to free some memory on mybinder.org
data = nothing
GC.gc()

### Types are values

Types can also be passed around as values and bound to variables with no special rules. This makes implementing factories or constructors easy:

In [ ]:
zeros(Float64, 3, 3)

### Macros

A macro is written just like a normal Julia function. The difference is that a macro operates on the *expression* itself, not on its value:

`@show` : print out the *name* of a variable and its value. Great for quick debugging:

In [ ]:
x = 5
@show x

`@time` measure the elapsed time of an expression and return the result of that expression:

In [ ]:
@time sqrt(big(π))

We have seen its sibling `@benchmark` above.

# What's hard to do in Julia?

- What is the compiler team working on making better?
- What are some subtle problems that the Julia team would like to fix?

[![What's bad about Julia talk](img/bad_about_julia.png)](https://www.youtube.com/watch?v=TPuJsgyu87U)

## Compiler latency

- The JIT compiler runs each time it sees a function being called with a new input type
- That makes the first call slow, since you have to wait for the JIT
  - This makes Julia awkward for things like shell scripts or AWS lambda
- The compiler is essentially serial

## Static compilation

* To avoid the JIT lag, you can compile a Julia package to a standalone executable using [PackageCompiler.jl](https://github.com/JuliaLang/PackageCompiler.jl), but:
    * This workflow is not used widely
    * The resulting libraries tend to be quite large
    * Some people have compiled Julia code for Arduino

## Static analysis

* There are some linting tools for Julia (like the `vscode-julia` extension for Visual Studio Code), but they are not as mature as languages like Python, C, Java, etc.
* Static analysis of Julia is harder, since the language itself is dynamically typed
  * To be fair, static analysis of C++ is [undecidable](https://blog.reverberate.org/2013/08/parsing-c-is-literally-undecidable.html) but we still have tools that do a pretty good job most of the time

## HPC for simulation science (in contrast to data science)

- The ecosystem for traditional HPC frameworks needs to be advanced
- Binaries (depending on MPI) must still be configured manually
- Some momemtum comes from the [CliMA project at CalTech](https://clima.caltech.edu/)
- We are working on it with Trixi.jl

## Embedded computing

- It can be hard to run Julia on memory-limited systems, since you need the compiler living alongside your code
- Static compilation can help, but this isn't a well-developed workflow yet

## Summary

- Teaching
  - Interactive notebooks (Jupyter, Pluto)
  - Simple setup and installation
  - Multiple dispatch "feels mathematically correct"
- Open source development
  - Avoids two-language problem
  - Best practices built into the language
  - Great community of Julia users and experts willing to help
- Research
  - Open source
  - Reproducibility
  - Young but relatively rich scientific ecosystem

# Useful Julia Tools

## Julia VSCode

https://github.com/julia-vscode/julia-vscode

* Code highlighting, snippets, linting, and completions
* Integrated plot and table viewers
* General extension support via the VSCode language server

<div align="center">
  <img src="https://github.com/julia-vscode/julia-vscode.github.io/raw/243947a5d00b47ff65b9c133da287697f84aeada/img/newscreen5.png" width="70%"/>
</div>

## [Revise.jl](https://github.com/timholy/Revise.jl)

- Reduce overhead while developing
- Partial replacement of `make`

<div align="center">
  <img src="https://github.com/timholy/Revise.jl/raw/master/images/revise-wordmark.png" width="60%">
</div>

## [DifferentialEquations/OrdinaryDiffEq](https://github.com/SciML/DifferentialEquations.jl)

* { ordinary | delay | stochastic | * } differential equations
* Automatic differentiation and sparsity detection
* Sensitivity analysis and parameter estimation
* Access to pure-Julia solvers and existing C and Fortran solvers

<div align="center">
  <img src="https://github.com/SciML/DifferentialEquations.jl/raw/master/assets/DifferentialEquations_Example.png" width="50%"/>
</div>


## [DataFrames](https://github.com/JuliaData/DataFrames.jl)

- In-memory tabular data
- Joining, indexing, grouping, and split-apply-combine

```julia
julia> using DataFrames

julia> df = DataFrame(A = 1:4, B = ["M", "F", "F", "M"])
4×2 DataFrame
│ Row │ A     │ B      │
│     │ Int64 │ String │
├─────┼───────┼────────┤
│ 1   │ 1     │ M      │
│ 2   │ 2     │ F      │
│ 3   │ 3     │ F      │
│ 4   │ 4     │ M      │
```

## Automatic differentiation

- Forward/reverse mode AD and more
  - [ForwardDiff](https://github.com/JuliaDiff/ForwardDiff.jl)
  - [ReverseDiff](https://github.com/JuliaDiff/ReverseDiff.jl)
  - [Zygote](https://github.com/FluxML/Zygote.jl)
  - More to come
- Common tools in [ChainRules](https://github.com/JuliaDiff/ChainRules.jl)

<div align="center">
  <img src="https://avatars.githubusercontent.com/u/7750915?s=200&v=4" width="20%">
</div>

## [LoopVectorization.jl](https://github.com/JuliaSIMD/LoopVectorization.jl), [Polyester.jl](https://github.com/JuliaSIMD/CheapThreads.jl)

- Advanced SIMD programming in Julia
- Able to beat MKL with pure Julia code

<div align="center">
  <img src="https://avatars.githubusercontent.com/u/80543003?s=200&v=4" width="20%">
</div>

## [JuliaGPU](https://juliagpu.org/)

- Compile Julia on GPUs
- High-level interface used for ML, data science, differential equations, ...

<div align="center">
  <img src="https://github.com/JuliaGPU/juliagpu.org/raw/master/_assets/logo_crop.png" width="20%">
</div>